In [100]:
from openai import OpenAI

client = OpenAI()

## Tools to get the current temperature for a given location

In [101]:

tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get the current temperature for a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type":"string",
                    "description": "City and country eg Nairobi, Kenya"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        }
    }
]

In [102]:
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "user",
            "content": "What is the weather in Nairobi"
        }
    ],
    tools=tools
)

In [103]:
print(response.output[0].model_dump_json(indent=2))

{
  "arguments": "{\"location\":\"Nairobi, Kenya\"}",
  "call_id": "call_XRNHsNJpt3y1tn3Kc7LeDT9A",
  "name": "get_weather",
  "type": "function_call",
  "id": "fc_681e6776af448191b4602941461df9c80b9da15853369622",
  "status": "completed"
}


## Tool to send an Email

In [104]:

tools = [
    {
        "type": "function",
        "name": "send_email",
        "description": "Send an email to a given recipient with a subject and message",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {
                    "type": "string",
                    "description": "The recipient of the email address",
                },
                "subject": {"type": "string", "description": "Email subject line"},
                "body": {"type": "string", "description": "Body of the email message"},
            },
            "required": ["to", "subject", "body"],
            "additionalProperties": False,
        },
    }
]

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "user",
            "content": "Can you send an email to jmskanyiri@gmail.com and otuyaalvin@gmail.com saying hi",
        }
    ],
    tools=tools
)

In [105]:
for tool_call in response.output:
    print(tool_call.model_dump_json(indent=2))

{
  "arguments": "{\"to\":\"jmskanyiri@gmail.com\",\"subject\":\"Hello!\",\"body\":\"Hi!\"}",
  "call_id": "call_NKcrrO8byxqgCT1kTUZqPngh",
  "name": "send_email",
  "type": "function_call",
  "id": "fc_681e677829c88191b0d25f78a4f8e388031ef305ac9d2fea",
  "status": "completed"
}
{
  "arguments": "{\"to\":\"otuyaalvin@gmail.com\",\"subject\":\"Hello!\",\"body\":\"Hi!\"}",
  "call_id": "call_MQ4WuN4pWgLQs2U9elQMJsdY",
  "name": "send_email",
  "type": "function_call",
  "id": "fc_681e67787f8c81918514ced075f97e77031ef305ac9d2fea",
  "status": "completed"
}


## Function to search knowledge base~

In [106]:
tools = [
    {
        "type": "function",
        "name": "search_knowledge_base",
        "description": "Query a knowledge base to retrieve relevant info in a topic",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The user question or search query"
                },
                "options": {
                    "type": "object",
                    "properties": {
                        "num_results": {
                            "type": "number",
                            "description": "Number of top results to return"
                        },
                        "domain_filter": {
                            "type": ["string", "null"],
                            "description": "Optional domain to narrow the search eg finance, medical. Pass null if not needed"
                        },
                        "sort_by": {
                            "type": ["string", "null"],
                            "enum": ["relevance", "date", "popularity", "alphabetical"],
                            "description": "How to sort the results. Pass null if not needed"
                        }
                    },
                    "required": [
                        "num_results",
                        "domain_filter",
                        "sort_by"
                    ],
                    "additionalProperties": False
                }
            },
            "required": [
                "query",
                "options"
            ],
            "additionalProperties": False
        }
    }
]

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "user",
            "content": "Can you find information about ChatGPT in the AI knowledge base?"
        }
    ],
    tools=tools
)

In [107]:
for tool_call in response.output:
    print(tool_call.model_dump_json(indent=2))

{
  "arguments": "{\"query\":\"ChatGPT\",\"options\":{\"num_results\":5,\"domain_filter\":null,\"sort_by\":\"relevance\"}}",
  "call_id": "call_yaJthG3IQj3JjfJNap0Goj7O",
  "name": "search_knowledge_base",
  "type": "function_call",
  "id": "fc_681e677a1108819186fb1b302d243a6a0adda71e5014e567",
  "status": "completed"
}


## 🧠 Function Calling Steps

### Sample get_weather function that you have defined in your codebase

In [108]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


1. **Call the model**  
   Send the system message, user messages, and the list of available functions to the model.

In [109]:
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get the current temperature for provided coordinates in celsius",
        "strict": True,
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "number",
                },
                "longitude": {
                    "type": "number",
                },
            },
            "required": [
                "latitude",
                "longitude"
            ],
            "additionalProperties": False
        },
    }
]

input_message = [
    {
        "role": "user",
        "content": "What is the weather in Nairobi Kenya",
    }
]

response = client.responses.create(
    model="gpt-4o-mini",
    input=input_message,
    tools=tools
)

2. **Model decides to call a function**  
   The model returns the function name and input arguments.

In [110]:
response.output

[ResponseFunctionToolCall(arguments='{"latitude":-1.286389,"longitude":36.817223}', call_id='call_8e8Gcz258ooVYfe1Rl4wKM4g', name='get_weather', type='function_call', id='fc_681e677ba9488191b2ae90c8d4c834530ab54058d7dcd812', status='completed')]

3. **Execute the function**  
   Run the corresponding function code, parse the model's response, and handle the function call.


In [111]:
#Get the tool call
tool_call = response.output[0]
print(tool_call.model_dump_json(indent=2))

{
  "arguments": "{\"latitude\":-1.286389,\"longitude\":36.817223}",
  "call_id": "call_8e8Gcz258ooVYfe1Rl4wKM4g",
  "name": "get_weather",
  "type": "function_call",
  "id": "fc_681e677ba9488191b2ae90c8d4c834530ab54058d7dcd812",
  "status": "completed"
}


In [112]:
#Get the tool arguments
tool_call.arguments

'{"latitude":-1.286389,"longitude":36.817223}'

In [113]:
import json

#convert the json formatted string into a python dictionary
args = json.loads(tool_call.arguments)

args

{'latitude': -1.286389, 'longitude': 36.817223}

In [114]:
#Call the function passing the required arguments

results = get_weather(args['latitude'], args['longitude'])
results

17.5

4. **Return results to the model**  
   Send the results of the function back to the model so it can use them in its response.


In [115]:
#append model's function call message
input_message.append(tool_call)

#append result message
input_message.append(
    {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(results)
    }
)

response_2 = client.responses.create(
    model="gpt-4o-mini",
    input=input_message,
    tools=tools
)


5. **Model generates final response**  
   The model incorporates the function results into its output and sends a final response.


In [116]:
print(response_2.output_text)

The current temperature in Nairobi, Kenya, is 17.5°C. If you need more details about the weather, let me know!


## Handling function calls

In [117]:
#Functions

def get_weather(location: str):
    return 14

def send_email(to:str, subject:str, body:str):
    return f"Email sent to {to}"

In [123]:
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get the current temperature for a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country eg Nairobi, Kenya",
                }
            },
            "required": ["location"],
            "additionalProperties": False,
        },
    },
    {
        "type": "function",
        "name": "send_email",
        "description": "Send an email to a given recipient with a subject and message",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {
                    "type": "string",
                    "description": "The recipient of the email address",
                },
                "subject": {
                    "type": "string",
                    "description": "Email subject line",
                },
                "body": {
                    "type": "string",
                    "description": "Body of the email message",
                },
            },
            "required": ["to", "subject", "body"],
            "additionalProperties": False,
        },
    },
]

input_message = [
    {
        "role": "user",
        "content": "What is the weather in Nairobi Kenya. send an email to jmskanyiri@gmail.com and otuyaalvin@gmail.com saying hi",
    }
]

response = client.responses.create(
    model="gpt-4o-mini",
    input=input_message,
    tools=tools,
)

In [124]:
for tool_call in response.output:
    print(tool_call.model_dump_json(indent=2))

{
  "arguments": "{\"location\":\"Nairobi, Kenya\"}",
  "call_id": "call_061CDDvRQTMGdzqZzlwtzPxG",
  "name": "get_weather",
  "type": "function_call",
  "id": "fc_681e67b927fc8191ab29b7b13ba7a7820fa017c2d745b68a",
  "status": "completed"
}
{
  "arguments": "{\"to\":\"jmskanyiri@gmail.com\",\"subject\":\"Hello!\",\"body\":\"Hi!\"}",
  "call_id": "call_U7nNxcseWk94XAUbHHqjTn54",
  "name": "send_email",
  "type": "function_call",
  "id": "fc_681e67b967648191b607ddfd29dba41b0fa017c2d745b68a",
  "status": "completed"
}
{
  "arguments": "{\"to\":\"otuyaalvin@gmail.com\",\"subject\":\"Hello!\",\"body\":\"Hi!\"}",
  "call_id": "call_os5ywI4NZ0xrSZ30ZFYn8Krz",
  "name": "send_email",
  "type": "function_call",
  "id": "fc_681e67b9c468819187eec45eb3164c060fa017c2d745b68a",
  "status": "completed"
}


In [125]:
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)
    
    if name == "send_email":
        return send_email(**args)

In [126]:
for tool_call in response.output:
    
    #Get the tool name
    tool_name = tool_call.name
    print(tool_name)
    
    #Get the tool args
    tool_arguments = json.loads(tool_call.arguments)
    print(tool_arguments)
    
    input_message.append(tool_call)
    
    print("\n")
    
    tool_result = call_function(tool_name, tool_arguments)
    
    input_message.append(
        {
            "type": "function_call_output",
            "call_id": tool_call.call_id,
            "output": str(tool_result)
        }
    )
    

get_weather
{'location': 'Nairobi, Kenya'}


send_email
{'to': 'jmskanyiri@gmail.com', 'subject': 'Hello!', 'body': 'Hi!'}


send_email
{'to': 'otuyaalvin@gmail.com', 'subject': 'Hello!', 'body': 'Hi!'}




In [127]:
for message in input_message:
    print(message)

{'role': 'user', 'content': 'What is the weather in Nairobi Kenya. send an email to jmskanyiri@gmail.com and otuyaalvin@gmail.com saying hi'}
ResponseFunctionToolCall(arguments='{"location":"Nairobi, Kenya"}', call_id='call_061CDDvRQTMGdzqZzlwtzPxG', name='get_weather', type='function_call', id='fc_681e67b927fc8191ab29b7b13ba7a7820fa017c2d745b68a', status='completed')
{'type': 'function_call_output', 'call_id': 'call_061CDDvRQTMGdzqZzlwtzPxG', 'output': '14'}
ResponseFunctionToolCall(arguments='{"to":"jmskanyiri@gmail.com","subject":"Hello!","body":"Hi!"}', call_id='call_U7nNxcseWk94XAUbHHqjTn54', name='send_email', type='function_call', id='fc_681e67b967648191b607ddfd29dba41b0fa017c2d745b68a', status='completed')
{'type': 'function_call_output', 'call_id': 'call_U7nNxcseWk94XAUbHHqjTn54', 'output': 'Email sent to jmskanyiri@gmail.com'}
ResponseFunctionToolCall(arguments='{"to":"otuyaalvin@gmail.com","subject":"Hello!","body":"Hi!"}', call_id='call_os5ywI4NZ0xrSZ30ZFYn8Krz', name='send

In [128]:
input_message.append(
    {
        "role": "user",
        "content": "Give a summary of all the completed task"
    }
)
response_2 = client.responses.create(
    model="gpt-4o-mini",
    input=input_message,
    tools=tools
)

response_2.output_text

'Here’s a summary of the completed tasks:\n\n1. **Weather Check**: Retrieved the current temperature in Nairobi, Kenya, which is 14°C.\n2. **Emails Sent**:\n   - An email with the subject "Hello!" and message "Hi!" was sent to **jmskanyiri@gmail.com**.\n   - An email with the same subject and message was sent to **otuyaalvin@gmail.com**.'